## ***Nth-order Polynomial Linear Regression: An Introduction to Neural Networks*** ##



#### ***1. Motivation*** ####

Linear regression lies at the heart of neural networks. Although not even close to the pinnacle of neural networks' abilities, linear models are straightforward to analyze and can be studied as a starting point for understanding more complex, non-linear models. Fitting nth-order polynomials with linear regression can be used to introduce basic terminology and fundamental concepts in the field of deep learning.



#### ***2. Linear Regression*** ####

##### *2.1 Preliminaries on Fitting Polynomials* #####

An nth-order polynomial can be described as a linear combination of the powers of the independent variable from 0 to $n$. For example, the third-order polynomial

$$ y(x) = 1 + x + 3x^2 + 4x^3 $$

is described by the linear combination of the coefficients $(1, 1, 3, 4)$ multiplying the powers of the independent variable, $x$. In simple terms, fitting the polynomial with linear regression means estimating the coefficients. Knowing the order $n$ and ordered pairs $(x, y)$ of the polynomial, one could first randomly guess the coefficients and calculate the $y$-values that they produce. Based on the errors between the calculated and actual $y$-values, the coefficients could then be systematically corrected to yield a more accurate estimate. 



##### *2.2 Applicable Terminology in Neural Networks* #####

One characteristic of a neural network is the number of *layers*. The first layer is the *input layer*, which for polynomial fitting contains $x$-values from the domain. The last layer is the *output layer*, which contains *predicted* $\hat{y}$-values. Neural networks can have any number of *hidden layers* in between the input and output layers, but linear regression requires none and for the sake of simplicity, implementing hidden layers is for now omitted. 

Inter-layer computations must take the inputs from one layer to another. This process is described as *forward propagation*. For this example of linear regression, computations made on $x$-values from the input layer result in $\hat{y}$-values at the output layer. Here, the powers of the inputs are multiplied by the linear combination of the coefficients to get the outputs. In neural networks terminology, the coefficients for the powers of $x$ from $1$ to $n$ are the *weights*, and the coefficient multiplying $x^0$, or $1$, is the *bias*. For nth-order polynomial fitting with coefficients $\theta_j$ , the predicted outputs are calculated as 

$$ \hat{y_i} = \sum_{j=0}^n \theta_j \cdot x_i^j $$

where $i$ goes from $0$ to $N$, the number of input-output pairs $(x_i, y_i)$ training the neural network. 



##### *2.3 Gradient Descent and Mean Squared Error* #####

The intuitive approach of randomly initializing and systematically optimizing the weights and biases is implemented in many neural networks. The optimization occurs through *backpropagation*, as the error calculated at the output layer is propagated back to the weights that generated it. One prominent *cost* function for calculating the error is

$$ MSE = \frac{1}{2N} \sum_{i=1}^N (y_i - \hat{y_i})^2 $$

where the factor of $1/2$ is not necessary but is often included to simplify further calculations. Gradient descent is the algorithm used to update the weights and bias ($\theta_0$) accordig to

$$ \theta_j = \theta_j - \alpha \frac{\partial J}{\partial \theta_j} $$

where $J$ is the cost function and $\alpha$ is the *learning rate*, often starting at $0.01$, but whose value depends on the model and can range from $0.0$ to $1.0$. When fitting polynomials, $j$ goes from $0$ to $n$ such that all the coefficients are updated to minimize the error. Since $\hat{y}$ depends on the weights, chain rule is applied when computing the gradient. For the simple example here, the gradient for each $\theta_j$ is calculated as 

$$  \frac{1}{N} \sum_{i=1}^N(y_i - \hat{y_i}) \cdot x_i^j $$


#### ***3. Numpy Linear Regressor*** ####

The model implemented below uses previously discussed concepts to fit nth-order polynomials. The inputs are initialized in a one-dimensional array, and the outputs are an nth-order polynomial of x. Since the summations above can be simplified using matrix multiplication, the data and weights are stored as matrices for easier computations. 

In [230]:
import numpy as np

class Model:
    def __init__(self, x, y, n):
        self.y = y
        self.n = n
        self.input_size = len(x)

        # (n+1) x (len(x)) matrix with x^0, x^1, ..., x^n
        self.x_matrix = np.ones(self.input_size)
        for i in range(1, self.n+1):
            self.x_matrix = np.vstack((self.x_matrix, np.power(x, i)))

        # 1 x (n+1) matrix with weights (weights[0] is the bias)
        rng = np.random.default_rng()
        self.weights = rng.random((1, self.n+1))

    def train(self, num_epochs=5, lr=0.01, t=1000):
        for epoch in range(num_epochs):
            for run in range(t):
                # compute predicted values based on current weights
                y_pred = np.matmul(self.weights, self.x_matrix)
                # backpropagate the errors
                e = y_pred - self.y
                if run % 100 == 99:
                    # log MSE every 100 runs
                    print(f"Run {run}: {1/(2*self.input_size)*np.dot(e[0,:], e[0,:])}")
                grad_y_pred = 1/self.input_size*e
                self.weights -= lr*np.matmul(grad_y_pred, np.transpose(self.x_matrix))
            # log the final weights
            print(f"\nEpoch {epoch+1}: {self.weights}\n")

''' third-order example '''
x = np.linspace(-3.14, 3.14, 100)
y = 1 + x + 3*x**2 + 4*x**3

third_order = Model(x, y, 3)
third_order.train()

Run 99: 0.01611062856201498
Run 199: 0.00563745743811019
Run 299: 0.0019744369242061265
Run 399: 0.0006922579862671777
Run 499: 0.00024302244943860706
Run 599: 8.544425265981266e-05
Run 699: 3.009532041193852e-05
Run 799: 1.062271419161289e-05
Run 899: 3.758841183008195e-06
Run 999: 1.3339450980219476e-06

Epoch 1: [[0.99920197 1.00209686 3.00013513 3.99970702]]

Run 99: 4.7499985397807514e-07
Run 199: 1.6980423917039367e-07
Run 299: 6.097489724335581e-08
Run 399: 2.2007225843138103e-08
Run 499: 7.988509007589245e-09
Run 599: 2.9182947151672223e-09
Run 699: 1.0735551567461952e-09
Run 799: 3.979241298820601e-10
Run 899: 1.4868704391301974e-10
Run 999: 5.6029152797107894e-11

Epoch 2: [[0.99998951 1.00001076 3.00000178 3.9999985 ]]

Run 99: 2.1297977742968316e-11
Run 199: 8.167664285701047e-12
Run 299: 3.1599291236471135e-12
Run 399: 1.2330920908022253e-12
Run 499: 4.851886812599511e-13
Run 599: 1.9241094845364487e-13
Run 699: 7.68635187755376e-14
Run 799: 3.0911682571600885e-14
Run 899:


#### ***4. Taking It Further: Hidden Layers*** ####

The above example can hardly be called a neural network, as it has no hidden layers! Adding hidden layers involves making several updates to the model. In the same way that weights in the previous example have been used to propagate the input to the output, more weights can be used to propagate the input through a network of hidden layers to reach the output. However, simply adding more layers does not add much value to the model, as the computations would still be linear. In order to create more complex models, a non-linear *activation function* is applied to each hidden layer. Implementing hidden layers requires additional care during the backpropagation steps, as the layers before and after activation as well as the derivative of the activation function must be taken into account when computing the gradient. 